In [31]:
    import pandas as pd
    from datetime import time
    from sys import path
    
    path.append('/Users/carrey/Work/KDD2017')

In [32]:
    #Load Data Sets
    pd_links = pd.read_csv('../../dataSets/training/links (table 3).csv')
    pd_routes = pd.read_csv('../../dataSets/training/routes (table 4).csv')
    pd_trajectories = pd.read_csv('../../dataSets/training/trajectories(table 5)_training.csv')
    pd_volume = pd.read_csv('../../dataSets/training/volume(table 6)_training.csv')

In [33]:
    #先处理trajectories数据
    pd_trajectories.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98


In [34]:
    #字符转为时间格式
    pd_trajectories['starting_time'] = pd.to_datetime(pd_trajectories['starting_time'], format = '%Y-%m-%d %H:%M:%S')
    
    #获取日期
    pd_trajectories['date'] = pd_trajectories['starting_time'].dt.date
    
    #获取时间
    pd_trajectories['time'] = pd_trajectories['starting_time'].dt.time
    
    #获取小时
    def get_hour(t):
        return t.hour
    
    pd_trajectories['hour'] = pd_trajectories['time'].apply(get_hour)
    
    #获取时间段
    def get_timewindow(t):
        delta = 20
        if t.minute < delta:
            window = [time(t.hour, 0), time(t.hour,20)]
        elif t.minute < delta*2:
            window = [time(t.hour, 20), time(t.hour, 40)]
        else:
            try:
                window = [time(t.hour, 40), time(t.hour + 1, 0)]
            except ValueError:
                window = [time(t.hour, 40), time(0,0,0)]
        s_window = '[' + str(window[0]) + ',' + str(window[1]) + ')'
        return s_window
    
    def get_weatherhour(t):
        delta = 3
        k = int(t.hour)/3
        hour = k*3
        return hour
    
    pd_trajectories['time_window'] = pd_trajectories['time'].apply(get_timewindow)
    
    #获取星期
    pd_trajectories['weekday'] = pd_trajectories['starting_time'].dt.dayofweek + 1  #pandas的周一从0开始，周日为6
    
    #获取是否为法定假日(国庆和中秋）
    holiday1 = [pd.Timestamp('2016-10-1'), pd.Timestamp('2016-10-8')]
    holiday2 = [pd.Timestamp('2016-9-15'), pd.Timestamp('2016-9-18')]
    
    pd_trajectories['holiday'] = pd_trajectories['starting_time'].between(holiday1[0],holiday1[1])\
                                    | pd_trajectories['starting_time'].between(holiday2[0],holiday2[1])
    
    #获取weather_hour便于和weather联结
    pd_trajectories['weather_hour'] = pd_trajectories['starting_time'].apply(get_weatherhour)
    
    #删除'time'字段
    del pd_trajectories['time']
    
    pd_trajectories.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time,date,hour,time_window,weekday,holiday,weather_hour
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85,2016-07-19,0,"[00:00:00,00:20:00)",2,False,0
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98,2016-07-19,0,"[00:40:00,01:00:00)",2,False,0


In [35]:
    #获取links
    def get_links(seq):
        l_seq = seq.split(';')
        links = []
        for s in l_seq:
            link = s.split('#')[0]
            links.append(link)
        s_links = ';'.join(links)
        return s_links
    
    pd_trajectories['links'] = pd_trajectories['travel_seq'].apply(get_links)

In [36]:
    type(pd_trajectories['date'][0])

datetime.date

In [37]:
    pd_trajectories.to_csv('../../Processed_data/by_weikai/processed_train_trajectories.csv', index = False)

In [38]:
    pd_volume.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-09-19 23:09:25,2,0,1,0,NaN
1,2016-09-19 23:11:53,2,0,1,0,NaN
2,2016-09-19 23:13:54,2,0,1,0,NaN
3,2016-09-19 23:17:48,1,0,1,1,NaN
4,2016-09-19 23:16:07,2,0,1,0,NaN


In [39]:
    #处理volume数据
    
    #字符转为时间格式
    pd_volume['time'] = pd.to_datetime(pd_volume['time'], format = '%Y-%m-%d %H:%M:%S')
    
    #获取日期
    pd_volume['date'] = pd_volume['time'].dt.date
    
    #获取时间
    pd_volume['t'] = pd_volume['time'].dt.time
    
    #获取小时
    pd_volume['hour'] = pd_volume['t'].apply(get_hour)
    
    #获取时间段
    pd_volume['time_window'] = pd_volume['t'].apply(get_timewindow)
    
    #获取星期
    pd_volume['weekday'] = pd_volume['time'].dt.dayofweek + 1
    
    #获取是否为法定假日
    pd_volume['holiday'] = pd_volume['time'].between(holiday1[0],holiday1[1])\
                            | pd_volume['time'].between(holiday2[0],holiday2[1])
        
    #获取weather_hour，便于和weather联结
    pd_volume['weather_hour'] = pd_volume['time'].apply(get_weatherhour)
        
    #删除‘t'字段
    del pd_volume['t']
    
    pd_volume.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,date,hour,time_window,weekday,holiday,weather_hour
0,2016-09-19 23:09:25,2,0,1,0,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21
1,2016-09-19 23:11:53,2,0,1,0,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21
2,2016-09-19 23:13:54,2,0,1,0,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21
3,2016-09-19 23:17:48,1,0,1,1,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21
4,2016-09-19 23:16:07,2,0,1,0,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21


In [40]:
    #整理好的数据写入文件
    pd_volume.to_csv('../../Processed_data/by_weikai/processed_train_volume.csv', index = False)

In [41]:
    #处理link数据
    
    #每段link是否有岔路口
    def is_forkroad(in_out_top):
        try:
            n = in_out_top.split(',')
            if len(n) >1:
                return True
            else:
                return False
        except AttributeError:
            return False
    
    pd_links['fork'] = pd_links['in_top'].apply(is_forkroad) | pd_links['out_top'].apply(is_forkroad)
    pd_links.head()

,link_id,length,width,lanes,in_top,out_top,lane_width,fork
0,100,58,3,1,105,111,3,False
1,101,84,3,1,116,121,3,False
2,102,131,9,3,115,109,3,False
3,103,23,12,4,111,"122,116",3,True
4,104,293,9,3,109,112,3,False


In [42]:
    #处理好的link写入文件
    pd_links.to_csv('../../Processed_data/by_weikai/porcessed_train_link.csv', index = False)

In [43]:
    #处理routes数据
    
    #获取路径中最小宽度
    def get_min_width(links):
        l_links = links.strip().split(',')
        widths = []
        for link in l_links:
            k = int(link)
            i = pd_links[pd_links['link_id'] == k].index
            width = int(pd_links.ix[i,'width'])
            widths.append(width)
        return min(widths)
    
    pd_routes['min_width'] = pd_routes['link_seq'].apply(get_min_width)
    pd_routes.head()

,intersection_id,tollgate_id,link_seq,min_width
0,A,2,"110,123,107,108,120,117",3
1,A,3,"110,123,107,108,119,114,118,122",3
2,B,1,"105,100,111,103,116,101,121,106,113",3
3,B,3,"105,100,111,103,122",3
4,C,1,"115,102,109,104,112,111,103,116,101,121,106,113",3


In [44]:
    #获取路线总长度
    def get_length(links):
        l_links = links.strip().split(',')
        length = 0
        for link in l_links:
            k = int(link)
            i = pd_links[pd_links['link_id'] == k].index
            l = int(pd_links.ix[i, 'length'])
            length += l
        return length
    pd_routes['length'] = pd_routes['link_seq'].apply(get_length)
    pd_routes.head()

,intersection_id,tollgate_id,link_seq,min_width,length
0,A,2,"110,123,107,108,120,117",3,384
1,A,3,"110,123,107,108,119,114,118,122",3,852
2,B,1,"105,100,111,103,116,101,121,106,113",3,821
3,B,3,"105,100,111,103,122",3,477
4,C,1,"115,102,109,104,112,111,103,116,101,121,106,113",3,1550


In [45]:
    #处理好的数据写入文件
    pd_routes.to_csv('../../Processed_data/by_weikai/processed_train_routes.csv', index = False)

In [46]:
    #处理weather数据
    pd_weather = pd.read_csv('../../dataSets/training/weather (table 7)_training_update.csv')
    pd_weather['date'] = pd.to_datetime(pd_weather['date'],format = '%Y-%m-%d')
    pd_weather['date'] = pd_weather['date'].dt.date
    pd_weather.rename(index = str, columns = {'hour':'weather_hour'}, inplace = True)
    pd_weather.head()

,date,weather_hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-07-01,0,1000.4,1005.3,225.0,2.1,26.4,94.0,0.0
1,2016-07-01,3,1000.5,1005.3,187.0,2.7,29.0,76.0,0.0
2,2016-07-01,6,998.9,1003.7,212.0,2.9,31.7,67.0,0.0
3,2016-07-01,9,998.7,1003.5,244.0,2.7,31.6,59.0,0.0
4,2016-07-01,12,999.7,1004.5,222.0,1.3,29.9,68.0,0.0


In [47]:
    #写入数据
    pd_weather.to_csv('../../Processed_data/by_weikai/processed_train_weather.csv', index = False)

In [48]:
    big_train_trajectorise = pd.merge(pd_trajectories, pd_weather, how = 'left', on = ['date','weather_hour'])
    big_train_trajectorise.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time,date,hour,time_window,weekday,holiday,weather_hour,links,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85,2016-07-19,0,"[00:00:00,00:20:00)",2,False,0,105;100;111;103;122,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,105;100;111;103;122,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,105;100;111;103;116;101;121;106;113,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05,2016-07-19,0,"[00:20:00,00:40:00)",2,False,0,110;123;107;108;120;117,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98,2016-07-19,0,"[00:40:00,01:00:00)",2,False,0,105;100;111;103;116;101;121;106;113,1000.9,1005.8,219.0,3.3,27.5,81.0,0.0


In [49]:
    big_train_trajectorise.to_csv('../../Processed_data/by_weikai/big_train_trajectorise.csv', index = False)

In [50]:
    big_train_volume = pd.merge(pd_volume, pd_weather, how = 'left', on = ['date','weather_hour'])
    big_train_volume.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,date,hour,time_window,weekday,holiday,weather_hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-09-19 23:09:25,2,0,1,0,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21,1012.0,1017.0,319.0,2.7,19.1,75.0,0.0
1,2016-09-19 23:11:53,2,0,1,0,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21,1012.0,1017.0,319.0,2.7,19.1,75.0,0.0
2,2016-09-19 23:13:54,2,0,1,0,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21,1012.0,1017.0,319.0,2.7,19.1,75.0,0.0
3,2016-09-19 23:17:48,1,0,1,1,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21,1012.0,1017.0,319.0,2.7,19.1,75.0,0.0
4,2016-09-19 23:16:07,2,0,1,0,NaN,2016-09-19,23,"[23:00:00,23:20:00)",1,False,21,1012.0,1017.0,319.0,2.7,19.1,75.0,0.0


In [51]:
    big_train_volume.to_csv('../../Processed_data/by_weikai/big_train_volume.csv', index = False)

In [52]:
    pd_test_trajectories = pd.read_csv('../../dataSets/testing_phase1/trajectories(table 5)_test1.csv')
        #字符转为时间格式
    pd_test_trajectories['starting_time'] = pd.to_datetime(pd_test_trajectories['starting_time'], format = '%Y-%m-%d %H:%M:%S')
    
    #获取日期
    pd_test_trajectories['date'] = pd_test_trajectories['starting_time'].dt.date
    
    #获取时间
    pd_test_trajectories['time'] = pd_test_trajectories['starting_time'].dt.time
    
    #获取小时
    def get_hour(t):
        return t.hour
    
    pd_test_trajectories['hour'] = pd_test_trajectories['time'].apply(get_hour)
    
    #获取时间段
    def get_timewindow(t):
        delta = 20
        if t.minute < delta:
            window = [time(t.hour, 0), time(t.hour,20)]
        elif time.minute < delta*2:
            window = [time(t.hour, 20), time(t.hour, 40)]
        else:
            try:
                window = [time(t.hour, 40), time(t.hour + 1, 0)]
            except ValueError:
                window = [time(t.hour, 40), time(0,0,0)]
        s_window = '[' + str(window[0]) + ',' + str(window[1]) + ')'
        return s_window
    
    def get_weatherhour(t):
        delta = 3
        k = int(t.hour)/3
        hour = k*3
        return hour
    
    pd_test_trajectories['time_window'] = pd_test_trajectories['time'].apply(get_timewindow)
    
    #获取星期
    pd_test_trajectories['weekday'] = pd_test_trajectories['starting_time'].dt.dayofweek + 1  #pandas的周一从0开始，周日为6
    
    #获取是否为法定假日(国庆和中秋）
    holiday1 = [pd.Timestamp('2016-10-1'), pd.Timestamp('2016-10-8')]
    holiday2 = [pd.Timestamp('2016-9-15'), pd.Timestamp('2016-9-18')]
    
    pd_test_trajectories['holiday'] = pd_test_trajectories['starting_time'].between(holiday1[0],holiday1[1])\
                                    | pd_test_trajectories['starting_time'].between(holiday2[0],holiday2[1])
    
    #获取weather_hour便于和weather联结
    pd_test_trajectories['weather_hour'] = pd_test_trajectories['starting_time'].apply(get_weatherhour)
    
    #删除'time'字段
    del pd_test_trajectories['time']
    
    #获取links
    def get_links(seq):
        l_seq = seq.split(';')
        links = []
        for s in l_seq:
            link = s.split('#')[0]
            links.append(link)
        s_links = ';'.join(links)
        return s_links
    
    pd_test_trajectories['links'] = pd_test_trajectories['travel_seq'].apply(get_links)
    pd_test_trajectories.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time,date,hour,time_window,weekday,holiday,weather_hour,links
0,A,2,1026631,2016-10-18 06:00:14,110#2016-10-18 06:00:14#7.65;123#2016-10-18 06...,27.54,2016-10-18,6,"[06:00:00,06:20:00)",2,False,6,110;123;107;108;120;117
1,C,3,1017312,2016-10-18 06:00:28,115#2016-10-18 06:00:28#9.35;102#2016-10-18 06...,139.53,2016-10-18,6,"[06:00:00,06:20:00)",2,False,6,115;102;109;104;112;111;103;122
2,A,2,1044671,2016-10-18 06:03:07,110#2016-10-18 06:03:07#7.22;123#2016-10-18 06...,26.01,2016-10-18,6,"[06:00:00,06:20:00)",2,False,6,110;123;107;108;120;117
3,A,3,1000017,2016-10-18 06:04:37,110#2016-10-18 06:04:37#7.39;123#2016-10-18 06...,78.40,2016-10-18,6,"[06:00:00,06:20:00)",2,False,6,110;123;107;108;119;114;118;122
4,A,2,1002850,2016-10-18 06:10:17,110#2016-10-18 06:10:17#14.33;123#2016-10-18 0...,71.00,2016-10-18,6,"[06:00:00,06:20:00)",2,False,6,110;123;107;108;120;117


In [53]:
    pd_test_trajectories.to_csv('../../Processed_data/by_weikai/processed_test_trajectories.csv', index = False)

In [54]:
    pd_test_volume = pd.read_csv('../../dataSets/testing_phase1/volume(table 6)_test1.csv')
    
    pd_test_volume.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-10-18 07:59:04,2,0,1,1,NaN
1,2016-10-18 07:59:31,2,0,1,1,NaN
2,2016-10-18 07:59:50,2,0,1,1,NaN
3,2016-10-18 07:32:33,3,0,1,1,NaN
4,2016-10-18 07:32:46,3,0,1,1,NaN


In [55]:
    #字符转为时间格式
    pd_test_volume['time'] = pd.to_datetime(pd_test_volume['time'], format = '%Y-%m-%d %H:%M:%S')
    
    #获取日期
    pd_test_volume['date'] = pd_test_volume['time'].dt.date
    
    #获取时间
    pd_test_volume['t'] = pd_test_volume['time'].dt.time
    
    #获取小时
    pd_test_volume['hour'] = pd_test_volume['t'].apply(get_hour)
    
    #获取时间段
    pd_test_volume['time_window'] = pd_test_volume['t'].apply(get_timewindow)
    
    #获取星期
    pd_test_volume['weekday'] = pd_test_volume['time'].dt.dayofweek + 1
    
    #获取是否为法定假日
    pd_test_volume['holiday'] = pd_test_volume['time'].between(holiday1[0],holiday1[1])\
                                | pd_test_volume['time'].between(holiday2[0],holiday2[1])
        
    #获取weather_hour，便于和weather联结
    pd_test_volume['weather_hour'] = pd_test_volume['time'].apply(get_weatherhour)
        
    #删除‘t'字段
    del pd_test_volume['t']
    
    pd_test_volume.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,date,hour,time_window,weekday,holiday,weather_hour
0,2016-10-18 07:59:04,2,0,1,1,NaN,2016-10-18,7,"[07:40:00,08:00:00)",2,False,6
1,2016-10-18 07:59:31,2,0,1,1,NaN,2016-10-18,7,"[07:40:00,08:00:00)",2,False,6
2,2016-10-18 07:59:50,2,0,1,1,NaN,2016-10-18,7,"[07:40:00,08:00:00)",2,False,6
3,2016-10-18 07:32:33,3,0,1,1,NaN,2016-10-18,7,"[07:40:00,08:00:00)",2,False,6
4,2016-10-18 07:32:46,3,0,1,1,NaN,2016-10-18,7,"[07:40:00,08:00:00)",2,False,6


In [56]:
    pd_test_volume.to_csv('../../Processed_data/by_weikai/processed_test_volume.csv', index = False)

In [57]:
    pd_test_weather = pd.read_csv('../../dataSets/testing_phase1/weather (table 7)_test1.csv')
    
    pd_test_weather['date'] = pd.to_datetime(pd_test_weather['date'],format = '%Y-%m-%d')
    pd_test_weather['date'] = pd_test_weather['date'].dt.date
    pd_test_weather.rename(index = str, columns = {'hour':'weather_hour'}, inplace = True)
    pd_test_weather.head()

,date,weather_hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-10-18,0,1015.2,1020.3,62.0,2.1,20.5,87.0,0.0
1,2016-10-18,3,1014.5,1019.5,37.0,2.5,23.3,73.0,0.0
2,2016-10-18,6,1012.5,1017.5,128.0,1.6,23.4,73.0,0.0
3,2016-10-18,9,1013.0,1018.0,205.0,0.6,20.9,94.0,1.8
4,2016-10-18,12,1013.8,1018.8,106.0,1.5,20.5,94.0,0.5


In [58]:
    pd_test_weather.to_csv('../../Processed_data/by_weikai/processed_test_weather.csv', index = False)

In [59]:
    big_test_trajectories = pd.merge(pd_test_trajectories, pd_test_weather, how = 'left', on = ['date','weather_hour'])
    big_test_volume = pd.merge(pd_test_volume, pd_test_weather, how = 'left', on = ['date','weather_hour'])

In [60]:
    big_test_trajectories.to_csv('../../Processed_data/by_weikai/big_test_trajectories.csv', index = False)
    big_test_volume.to_csv('../../Processed_data/by_weikai/big_test_volume.csv', index = False)